In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from pathlib import Path
from multiprocessing.dummy import Pool as ThreadPool
from collections import defaultdict
from natsort import natsorted
import tsfresh as tf
import sklearn

In [2]:
pd.set_option('max_columns', None)

### Data loading

In [3]:
import os
import glob

dataset_path = 'data_feats_90_c/'
csv_files = glob.glob(dataset_path+'*.csv')

all_df = []

for filename in csv_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    all_df.append(df)

df = pd.concat(all_df, axis=0, ignore_index=True)
df.head(1)

,Unnamed: 0,P-PDG__mean,P-PDG__variance,P-PDG__skewness,P-PDG__kurtosis,"P-PDG__fft_aggregated__aggtype_""centroid""","P-PDG__fft_aggregated__aggtype_""variance""","P-PDG__fft_aggregated__aggtype_""skew""","P-PDG__fft_aggregated__aggtype_""kurtosis""",P-PDG__maximum,P-PDG__minimum,P-PDG__median,P-PDG__quantile__q_0.1,P-PDG__quantile__q_0.2,P-PDG__quantile__q_0.3,P-PDG__quantile__q_0.4,P-PDG__quantile__q_0.6,P-PDG__quantile__q_0.7,P-PDG__quantile__q_0.8,P-PDG__quantile__q_0.9,P-PDG__variation_coefficient,P-PDG__mean_change,P-PDG__mean_second_derivative_central,P-PDG__friedrich_coefficients__coeff_1__m_3__r_30,P-PDG__friedrich_coefficients__coeff_3__m_3__r_30,P-TPT__mean,P-TPT__variance,P-TPT__skewness,P-TPT__kurtosis,"P-TPT__fft_aggregated__aggtype_""centroid""","P-TPT__fft_aggregated__aggtype_""variance""","P-TPT__fft_aggregated__aggtype_""skew""","P-TPT__fft_aggregated__aggtype_""kurtosis""",P-TPT__maximum,P-TPT__minimum,P-TPT__median,P-TPT__quantile__q_0.1,P-TPT__quantile__q_0.2,P-TPT__quantile__q_0.3,P-TPT__quantile__q_0.4,P-TPT__quantile__q_0.6,P-TPT__quantile__q_0.7,P-TPT__quantile__q_0.8,P-TPT__quantile__q_0.9,P-TPT__variation_coefficient,P-TPT__mean_change,P-TPT__mean_second_derivative_central,P-TPT__friedrich_coefficients__coeff_1__m_3__r_30,P-TPT__friedrich_coefficients__coeff_3__m_3__r_30,T-TPT__mean,T-TPT__variance,T-TPT__skewness,T-TPT__kurtosis,"T-TPT__fft_aggregated__aggtype_""centroid""","T-TPT__fft_aggregated__aggtype_""variance""","T-TPT__fft_aggregated__aggtype_""skew""","T-TPT__fft_aggregated__aggtype_""kurtosis""",T-TPT__maximum,T-TPT__minimum,T-TPT__median,T-TPT__quantile__q_0.1,T-TPT__quantile__q_0.2,T-TPT__quantile__q_0.3,T-TPT__quantile__q_0.4,T-TPT__quantile__q_0.6,T-TPT__quantile__q_0.7,T-TPT__quantile__q_0.8,T-TPT__quantile__q_0.9,T-TPT__variation_coefficient,T-TPT__mean_change,T-TPT__mean_second_derivative_central,T-TPT__friedrich_coefficients__coeff_1__m_3__r_30,T-TPT__friedrich_coefficients__coeff_3__m_3__r_30,P-MON-CKP__mean,P-MON-CKP__variance,P-MON-CKP__skewness,P-MON-CKP__kurtosis,"P-MON-CKP__fft_aggregated__aggtype_""centroid""","P-MON-CKP__fft_aggregated__aggtype_""variance""","P-MON-CKP__fft_aggregated__aggtype_""skew""","P-MON-CKP__fft_aggregated__aggtype_""kurtosis""",P-MON-CKP__maximum,P-MON-CKP__minimum,P-MON-CKP__median,P-MON-CKP__quantile__q_0.1,P-MON-CKP__quantile__q_0.2,P-MON-CKP__quantile__q_0.3,P-MON-CKP__quantile__q_0.4,P-MON-CKP__quantile__q_0.6,P-MON-CKP__quantile__q_0.7,P-MON-CKP__quantile__q_0.8,P-MON-CKP__quantile__q_0.9,P-MON-CKP__variation_coefficient,P-MON-CKP__mean_change,P-MON-CKP__mean_second_derivative_central,P-MON-CKP__friedrich_coefficients__coeff_1__m_3__r_30,P-MON-CKP__friedrich_coefficients__coeff_3__m_3__r_30,T-JUS-CKP__mean,T-JUS-CKP__variance,T-JUS-CKP__skewness,T-JUS-CKP__kurtosis,"T-JUS-CKP__fft_aggregated__aggtype_""centroid""","T-JUS-CKP__fft_aggregated__aggtype_""variance""","T-JUS-CKP__fft_aggregated__aggtype_""skew""","T-JUS-CKP__fft_aggregated__aggtype_""kurtosis""",T-JUS-CKP__maximum,T-JUS-CKP__minimum,T-JUS-CKP__median,T-JUS-CKP__quantile__q_0.1,T-JUS-CKP__quantile__q_0.2,T-JUS-CKP__quantile__q_0.3,T-JUS-CKP__quantile__q_0.4,T-JUS-CKP__quantile__q_0.6,T-JUS-CKP__quantile__q_0.7,T-JUS-CKP__quantile__q_0.8,T-JUS-CKP__quantile__q_0.9,T-JUS-CKP__variation_coefficient,T-JUS-CKP__mean_change,T-JUS-CKP__mean_second_derivative_central,T-JUS-CKP__friedrich_coefficients__coeff_1__m_3__r_30,T-JUS-CKP__friedrich_coefficients__coeff_3__m_3__r_30,QGL__mean,QGL__variance,QGL__skewness,QGL__kurtosis,"QGL__fft_aggregated__aggtype_""centroid""","QGL__fft_aggregated__aggtype_""variance""","QGL__fft_aggregated__aggtype_""skew""","QGL__fft_aggregated__aggtype_""kurtosis""",QGL__maximum,QGL__minimum,QGL__median,QGL__quantile__q_0.1,QGL__quantile__q_0.2,QGL__quantile__q_0.3,QGL__quantile__q_0.4,QGL__quantile__q_0.6,QGL__quantile__q_0.7,QGL__quantile__q_0.8,QGL__quantile__q_0.9,QGL__variation_coefficient,QGL__mean_change,QGL__mean_second_derivative_central,QGL__friedrich_c

In [4]:
df.drop(['Unnamed: 0'], axis='columns', inplace=True)      #leftover index column, unnecessary

df = df[df.class_code != 7]                                #paper does not use event class 7
df['class_code'].replace({8: 7}, inplace=True)


### Handling NA values

In [5]:
def print_na_sum(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        nans = df.isna().sum(axis=0)
        print(nans[nans!=0])

print_na_sum(df)

P-PDG__fft_aggregated__aggtype_"centroid"                 8571
P-PDG__fft_aggregated__aggtype_"variance"                 8571
P-PDG__fft_aggregated__aggtype_"skew"                    37244
P-PDG__fft_aggregated__aggtype_"kurtosis"                37244
P-PDG__variation_coefficient                              8571
P-PDG__friedrich_coefficients__coeff_1__m_3__r_30        20675
P-PDG__friedrich_coefficients__coeff_3__m_3__r_30        20675
P-TPT__fft_aggregated__aggtype_"centroid"                   64
P-TPT__fft_aggregated__aggtype_"variance"                   64
P-TPT__fft_aggregated__aggtype_"skew"                    30450
P-TPT__fft_aggregated__aggtype_"kurtosis"                30450
P-TPT__variation_coefficient                                64
P-TPT__friedrich_coefficients__coeff_1__m_3__r_30        12623
P-TPT__friedrich_coefficients__coeff_3__m_3__r_30        12623
T-TPT__fft_aggregated__aggtype_"centroid"                 6514
T-TPT__fft_aggregated__aggtype_"variance"              


Removing features with too many NA values

In [6]:
df.drop(['QGL__variation_coefficient'], axis='columns', inplace=True)
df.drop(list(df.filter(regex = 'friedrich')), axis = 1, inplace = True)
df.drop(list(df.filter(regex = 'fft_aggregated')), axis = 1, inplace = True)

In [7]:
print_na_sum(df)

P-PDG__variation_coefficient        8571
P-TPT__variation_coefficient          64
T-TPT__variation_coefficient        6514
P-MON-CKP__variation_coefficient    1200
T-JUS-CKP__variation_coefficient    1819
dtype: int64


Imputing leftover na values, for each class separately

In [8]:
imputed_df_list = []
for i in df['class_code'].unique():
    query = 'class_code == ' + str(i)
    imputed = tf.utilities.dataframe_functions.impute(df.query(query))
    imputed_df_list.append(imputed)
imputed_df = pd.concat(imputed_df_list)

/home/marko/.local/lib/python3.8/site-packages/pandas/core/generic.py:9032: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(cond, other, inplace, axis, level, errors=errors)


In [9]:
print_na_sum(imputed_df)

Series([], dtype: int64)


In [10]:
imputed_df = imputed_df.sample(frac=1).reset_index(drop=True)
sample_df = imputed_df#.sample(frac=0.01, random_state=42)
X = sample_df.iloc[:,:-1].to_numpy()
y = sample_df.iloc[:,-1].to_numpy()
sample_df.shape

(51764, 108)

In [11]:
X

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.23089100e+07,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 2.88168308e+07,  1.07427765e+03,  6.66213822e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.34674018e+07,  4.12645008e+07, -8.96772794e-01, ...,
         1.59378254e+00,  1.69290787e-03, -5.44318182e-05]])

In [12]:
y

array([0, 0, 5, ..., 6, 0, 3])

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
from sklearn.model_selection import KFold   

#todo: include window size into grid search

window_size_hp = [300, 600, 900]
regularization_hp = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]

In [15]:
from sklearn.model_selection import KFold   
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

kf = KFold(n_splits=5)

In [17]:
from sklearn.linear_model import SGDClassifier

for r in regularization_hp:
    i = 0
    for train_index, test_index in kf.split(X_train):
        X_k_train, X_k_test = X_train[train_index], X_train[test_index]
        y_k_train, y_k_test = y_train[train_index], y_train[test_index]
        scaler = StandardScaler().fit(X_k_train)
        X_k_train = scaler.transform(X_k_train)
        X_k_test = scaler.transform(X_k_test)
        log_reg = SGDClassifier(loss='log', max_iter=10000, alpha=r).fit(X_k_train, y_k_train)
        h = log_reg.predict(X_k_test)
        print('r='+str(r)+', i-th fold='+str(i)+', f1 score='+str(f1_score(y_k_test, h, average='weighted')))
        i += 1

r=1e-07, i-th fold=0, f1 score=0.8520103529440514
r=1e-07, i-th fold=1, f1 score=0.8792627838432763
r=1e-07, i-th fold=2, f1 score=0.8439397199429232
r=1e-07, i-th fold=3, f1 score=0.8779667171903017
r=1e-07, i-th fold=4, f1 score=0.8557572790373238
r=1e-06, i-th fold=0, f1 score=0.6602602087474834
r=1e-06, i-th fold=1, f1 score=0.7423437094527917
r=1e-06, i-th fold=2, f1 score=0.8548454894018055
r=1e-06, i-th fold=3, f1 score=0.8488636696166002
r=1e-06, i-th fold=4, f1 score=0.8441784748125547
r=1e-05, i-th fold=0, f1 score=0.8663370505976179
r=1e-05, i-th fold=1, f1 score=0.880112758632698
r=1e-05, i-th fold=2, f1 score=0.8986250626579495
r=1e-05, i-th fold=3, f1 score=0.8928673439643439
r=1e-05, i-th fold=4, f1 score=0.8836415953324852
r=0.0001, i-th fold=0, f1 score=0.8956395244389609
r=0.0001, i-th fold=1, f1 score=0.8931429424283753
r=0.0001, i-th fold=2, f1 score=0.8838254906601913
r=0.0001, i-th fold=3, f1 score=0.8833938421270617
r=0.0001, i-th fold=4, f1 score=0.8859646208788

KeyboardInterrupt: 

In [18]:
X_test = scaler.transform(X_test)
h = log_reg.predict(X_test)
f1_score(y_test, h, average='weighted')

0.8655504409453394

In [ ]:
log_reg = LogisticRegression(random_state=42).fit(X_train, y_train)
>>> clf.predict(X[:2, :])
array([0, 0])
>>> clf.predict_proba(X[:2, :])

clf.score(X, y)
0.97...